## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [1]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [2]:
with open("wordsim_similarity_goldstandard.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок.

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [3]:
for w1, w2 in list(score_map)[:2]:

  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))

  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: tiger-cat
Ground truth score: 7.35

Path: 0.091 
wup: 0.545 
shortest_path: 10.000

Words: tiger-tiger
Ground truth score: 10.00

Path: 1.000 
wup: 0.750 
shortest_path: 0.000


Вычисляем для всех пар несколько оценок

In [4]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list = [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)

    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

drink eat error: max() arg is an empty sequence
stock live error: max() arg is an empty sequence


In [5]:
wup_list[:3], true_list[:3], path_list[:3]

([0.9655172413793104, 1.0, 0.7272727272727273],
 [7.35, 10.0, 5.77],
 [0.5, 1.0, 0.14285714285714285])

## Вычисляем ранговую корреляцию Спирмена

In [6]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)


wup  Spearman R: 0.6438
path Spearman R: 0.6176


In [12]:
# Импортируем необходимые библиотеки и загружаем данные из файла CSV
import pandas as pd

# Загрузка файла CSV
file_path = 'Task_4_sample_7.csv'
data = pd.read_csv(file_path)

# Проверка содержимого файла
data.head()


,word_1,word_2,Score
0,direction,combination,2.25
1,start,match,4.47
2,hospital,infrastructure,4.63
3,century,year,7.59
4,gem,jewel,8.96


In [14]:
import nltk
from nltk.corpus import wordnet as wn
from scipy.stats import spearmanr

# Загрузка необходимых ресурсов NLTK
nltk.download('wordnet')

# Функция для нахождения максимальной близости с использованием заданной функции близости
def max_similarity(word1, word2, similarity_func):
    synsets1 = wn.synsets(word1, pos="n")
    synsets2 = wn.synsets(word2, pos="n")

    max_sim = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            try:
                sim = similarity_func(syn1, syn2)
                if sim is not None and sim > max_sim:
                    max_sim = sim
            except:
                continue
    return max_sim

# Вычисление близости по трем методам
data['path_sim'] = data.apply(lambda row: max_similarity(row['word_1'], row['word_2'], wn.path_similarity), axis=1)
data['lch_sim'] = data.apply(lambda row: max_similarity(row['word_1'], row['word_2'], wn.lch_similarity), axis=1)
data['wup_sim'] = data.apply(lambda row: max_similarity(row['word_1'], row['word_2'], wn.wup_similarity), axis=1)

data.head()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,word_1,word_2,Score,path_sim,lch_sim,wup_sim
0,direction,combination,2.25,0.142857,1.691676,0.666667
1,start,match,4.47,0.142857,1.691676,0.571429
2,hospital,infrastructure,4.63,0.083333,1.152680,0.266667
3,century,year,7.59,0.333333,2.538974,0.833333
4,gem,jewel,8.96,1.000000,3.637586,1.000000


In [15]:
data.head(20)

,word_1,word_2,Score,path_sim,lch_sim,wup_sim
0,direction,combination,2.25,0.142857,1.691676,0.666667
1,start,match,4.47,0.142857,1.691676,0.571429
2,hospital,infrastructure,4.63,0.083333,1.152680,0.266667
3,century,year,7.59,0.333333,2.538974,0.833333
4,gem,jewel,8.96,1.000000,3.637586,1.000000
5,life,term,4.50,0.333333,2.538974,0.857143
6,development,issue,3.97,0.250000,2.251292,0.750000
7,opera,performance,6.88,0.100000,1.335001,0.307692
8,student,professor,6.81,0.125000,1.558145,0.571429
9,stock,life,0.92,0.166667,1.845827,0.615385


In [17]:
from scipy.stats import spearmanr

coef, p = spearmanr(data['path_sim'], data['Score'])
print("path_sim Spearman R: %.4f" % coef)

path_sim Spearman R: 0.6142


In [18]:
coef, p = spearmanr(data['lch_sim'], data['Score'])
print("lch_sim  Spearman R: %.4f" % coef)

lch_sim  Spearman R: 0.6142


In [19]:
coef, p = spearmanr(data['wup_sim'], data['Score'])
print("wup  Spearman R: %.4f" % coef)

wup  Spearman R: 0.6384


In [22]:
# Проверка наличия ресурса WordNet и попытка выполнить задание с использованием этого ресурса
try:
    # Получение синсета для слова "journey" с пометкой noun и индексом 01
    journey_synset = wn.synset('journey.n.01')

    # Получение гипонимов для синсета
    hyponyms = journey_synset.hyponyms()

    # Подсчет количества гипонимов
    hyponyms_count = len(hyponyms)

    # Получение имени первого гипонима в списке
    first_hyponym_name = hyponyms[0].name() if hyponyms else 'No hyponyms'

    print("Количество гипонимов:", hyponyms_count)
    print("Имя первого гипонима:", first_hyponym_name)
except Exception as e:
    e


Количество гипонимов: 16
Имя первого гипонима: commute.n.01
